- # **SSL connection to DB2 on IBM Cloud**

- ### **Summary:**
1. [**Installing needed libraries**](#1-installing-libraries)
2. [**Importing environment variables**](#2-importing-environment-variables)
3. [**Creating connection object to the DB2 instance**](#3-creating-connection-object-to-the-db2-instance)
4. [**Running a sql view query with native ibm_db library**](#4-running-a-sql-view-query-with-native-ibm_db-library)
5. [**Defining functions to retrieve results in a List of Dictionary**](#5-defining-functions-to-retrieve-results-in-a-list-of-dictionary)
    - Criando uma lista de titulos de todas as tabelas da Instancia
    - Criando uma lista de titulos de tabelas a partir de um determinado filtro

    - Getting rows from a table view with the definned function
6. [**Converting the created Dictionary into a Pandas Dataframe**](#6-converting-the-created-dictionary-into-a-pandas-dataframe)

## **1. Installing libraries**

In [1]:
!pip install ipython-sql
!pip install ibm_db 
!pip install ibm_db_sa

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 1.6/1.6 MB 3.0 MB/s eta 0:00:00
  Using cached ipython_genutils-0.2.0-py2.py3-none-any.whl (26 kB)
     ---------------------------------------- 42.8/42.8 kB ? eta 0:00:00
     -------------------------------------- 190.9/190.9 kB 2.9 MB/s eta 0:00:00
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13705 sha256=3b28ecc5695f5a02036e173d829a0f567b6e9d083d8ff47407bdb536727e804b
  Stored in directory: c:\users\017924631\appdata\local\pip\cache\wheels\ec\32\b1\b9dda552a609c834467b28a174fa60d2c102e17857ccafd95b
Successfully built prettytable
     ---------------------------------------- 1.4/1.4 MB 6.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished w

## **2. Importing environment variables**

In [73]:
from dotenv import dotenv_values

config = dotenv_values(".env")
database, username, password, host, port = config["DATABASE"], config["USER"], config["PASSWORD"], config["HOST"], config["PORT"]

## **3. Creating connection object to the DB2 instance**

In [2]:
import ibm_db
#conn = ibm_db.connect("database","username","password") # cataloged database
conn = ibm_db.connect(f"DATABASE={database};HOSTNAME={host};PORT={port};PROTOCOL=TCPIP;UID={username};PWD={password};", "", "")

## **4. Running a sql view query with native ibm_db library**

In [74]:
sql_view = "SELECT * FROM MYTAB"
stmt_view = ibm_db.exec_immediate(conn, sql_view)
result = ibm_db.fetch_both(stmt_view)
print(result)

{'COL1': 0, 0: 0, 'COL2': '1', 1: '1', 'COL3': 0, 2: 0}


## **5. Defining functions to retrieve results in a List of Dictionary**

In [66]:
def results(command):
    """
    Commands to ibm_db never actually give you results. Instead, 
    you need to call one of the fetch methods on the command, 
    repeatedly, to get the results. This helper function to deal 
    with that.
    """

    
    from ibm_db import fetch_assoc

    ret = []
    result = fetch_assoc(command)
    while result:
        # This builds a list in memory. Theoretically, if there's a lot of rows,
        # we could run out of memory. In practice, I've never had that happen.
        # If it's ever a problem, you could use
        #     yield result
        # Then this function would become a generator. You lose the ability to access
        # results by index or slice them or whatever, but you retain
        # the ability to iterate on them.
        ret.append(result)
        result = fetch_assoc(command)
        #print(result)
    return ret  # Ditch this line if you choose to use a generator.


def filter_table_param(stack:list, param:str, filter:str):
    """
    This function receives a stack list of dicts and iteracts eache one 
    filtering a given parameter by a given filter string.
    """


    s = []
    for i in stack:
        if i[param] == filter:
            s.append(i)
    return s


- ### Criando uma lista de titulos de todas as tabelas da Instancia
- ### Criando uma lista de titulos de tabelas a partir de um determinado filtro

In [61]:
from ibm_db import tables

all_titles = results(tables(conn))
filtered_titles = filter_table_param(all_titles, 'TABLE_SCHEM', 'FXK17670')
print(filtered_titles)

[{'TABLE_CAT': None, 'TABLE_SCHEM': 'FXK17670', 'TABLE_NAME': 'MYTAB', 'TABLE_TYPE': 'TABLE', 'REMARKS': None}]


- ### Getting rows from a table view with the definned function

In [70]:
from ibm_db import exec_immediate

sql = 'SELECT * FROM ' + filtered_titles[0]['TABLE_NAME']  # Using our list of tables filtered_titles from before...
rows = results(exec_immediate(conn, sql))
print(rows)

## **6. Converting the created Dictionary into a Pandas Dataframe**

In [72]:
!pip install pandas
import pandas as pd

pd.DataFrame.from_dict(rows)

  Using cached pandas-1.5.2-cp310-cp310-win_amd64.whl (10.4 MB)
     ---------------------------------------- 14.8/14.8 MB 7.6 MB/s eta 0:00:00
     -------------------------------------- 499.4/499.4 kB 7.8 MB/s eta 0:00:00


,COL1,COL2,COL3
0,0,1,0
1,2,2,2
2,2,2,2
